# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to the developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [5]:
#!aws s3 cp   s3://msidorov.cs224/cs224u/data.tgz /home/ec2-user/data.tgz
#aws s3 cp   s3://msidorov.cs224/cs224u/bakeoff2/bakeoff-rel_ext-test-data.zip /home/ec2-user/bakeoff-rel_ext-test-data.zip
# unzip /home/ec2-user/bakeoff-rel_ext-test-data.zip /home/ec2-user/data/rel_ext_data/.
#!tar -xvf data.tgz   


#rel_ext_data_home = os.path.join('/home/ec2-user/data', 'rel_ext_data')
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [6]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [7]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [8]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [9]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [10]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [11]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [12]:
featurizers = [simple_bag_of_words_featurizer]

In [13]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [14]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.821      0.376      0.664        340       5716
author                    0.840      0.538      0.756        509       5885
capital                   0.486      0.189      0.370         95       5471
contains                  0.789      0.606      0.744       3904       9280
film_performance          0.800      0.573      0.741        766       6142
founders                  0.776      0.392      0.649        380       5756
genre                     0.707      0.171      0.434        170       5546
has_sibling               0.878      0.244      0.578        499       5875
has_spouse                0.890      0.327      0.662        594       5970
is_a                      0.655      0.221      0.470        497       5873
nationality               0.540      0.156      0.362        301       5677
parents     

Studying model weights might yield insights:

In [15]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.458 Córdoba
     2.400 Valais
     2.399 Taluks
     ..... .....
    -1.180 Asia
    -1.180 Iraq
    -1.871 22

Highest and lowest feature weights for relation author:

     3.012 author
     2.222 wrote
     2.215 by
     ..... .....
    -2.003 or
    -2.059 artist
    -2.811 superhero

Highest and lowest feature weights for relation capital:

     3.265 capital
     1.933 especially
     1.916 city
     ..... .....
    -1.311 also
    -1.312 new
    -1.458 Bronx

Highest and lowest feature weights for relation contains:

     2.841 third-largest
     2.078 tiny
     1.922 affiliated
     ..... .....
    -2.415 rise
    -2.900 Mile
    -9.202 Bronx

Highest and lowest feature weights for relation film_performance:

     4.209 starring
     3.598 co-starring
     3.398 alongside
     ..... .....
    -2.068 Tamil
    -3.946 Mohabbatein
    -3.985 double

Highest and lowest feature weights for relation founders:

     4.244

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [16]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [17]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

FileNotFoundError: [Errno 2] No such file or directory: 'data/glove.6B/glove.6B.300d.txt'

In [ ]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))                
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [17]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.872      0.459      0.739        340       5716
author                    0.827      0.442      0.704        509       5885
capital                   0.594      0.200      0.426         95       5471
contains                  0.660      0.415      0.591       3904       9280
film_performance          0.828      0.328      0.634        766       6142
founders                  0.768      0.226      0.519        380       5756
genre                     0.524      0.065      0.217        170       5546
has_sibling               0.838      0.238      0.558        499       5875
has_spouse                0.861      0.354      0.669        594       5970
is_a                      0.740      0.155      0.422        497       5873
nationality               0.615      0.213      0.446        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [ ]:
def run_svm_model_factory():
    
    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    
    model_factory = lambda: SVC(kernel='linear')
    baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)
    return baseline_results




In [19]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [20]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.802      0.382      0.658        340       5716
author                    0.782      0.607      0.740        509       5885
capital                   0.548      0.242      0.437         95       5471
contains                  0.773      0.596      0.730       3904       9280
film_performance          0.793      0.616      0.750        766       6142
founders                  0.742      0.447      0.656        380       5756
genre                     0.534      0.229      0.422        170       5546
has_sibling               0.801      0.242      0.549        499       5875
has_spouse                0.892      0.347      0.679        594       5970
is_a                      0.608      0.266      0.484        497       5873
nationality               0.474      0.183      0.359        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [18]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
results = rel_ext.experiment(
splits,
train_split='train',
test_split='dev',
featurizers=[directional_bag_of_words_featurizer]
)

print('Count of feature names: {count}'.format(count=len(results['vectorizer'].feature_names_)))



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.877      0.418      0.719        340       5716
author                    0.865      0.580      0.788        509       5885
capital                   0.550      0.232      0.431         95       5471
contains                  0.821      0.656      0.782       3904       9280
film_performance          0.834      0.650      0.789        766       6142
founders                  0.857      0.395      0.694        380       5756
genre                     0.721      0.259      0.531        170       5546
has_sibling               0.872      0.246      0.579        499       5875
has_spouse                0.895      0.345      0.679        594       5970
is_a                      0.753      0.239      0.527        497       5873
nationality               0.627      0.213      0.451        301       5677
parents     

In [19]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [20]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [21]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    combined = [start_symbol] + get_tags(s) + [end_symbol]
    return [combined[i] + ' ' + combined[i + 1] for i in range(len(combined) - 1)]



    
def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[middle_bigram_pos_tag_featurizer]
    )



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.808      0.347      0.639        340       5716
author                    0.735      0.322      0.585        509       5885
capital                   0.556      0.158      0.369         95       5471
contains                  0.751      0.599      0.715       3904       9280
film_performance          0.714      0.427      0.629        766       6142
founders                  0.604      0.168      0.398        380       5756
genre                     0.600      0.194      0.423        170       5546
has_sibling               0.701      0.164      0.424        499       5875
has_spouse                0.828      0.268      0.584        594       5970
is_a                      0.597      0.167      0.394        497       5873
nationality               0.432      0.063      0.199        301       5677
parents     

In [22]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [23]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [24]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word_synsets in get_synsets(ex.middle_POS):
            for syn in word_synsets:
                feature_counter[str(syn)] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word_synsets in get_synsets(ex.middle_POS):
            for syn in word_synsets:
                feature_counter[str(syn)] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    # list of synsets for each word
    synsets = [wn.synsets(w[0], convert_tag(w[1])) for w in wt]
    return synsets


    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None    


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[synset_featurizer]
    )


LookupError: 
**********************************************************************
  Resource [93mwordnet[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('wordnet')
  [0m
  Searched in:
    - '/home/ec2-user/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/ec2-user/anaconda3/envs/pytorch_p36/nltk_data'
    - '/home/ec2-user/anaconda3/envs/pytorch_p36/share/nltk_data'
    - '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/nltk_data'
**********************************************************************


In [ ]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():   
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [27]:
# Enter your system description in this cell.
# Please do not remove this comment.

# Description of the model:
# Model is uased bag_of_words (as it is provided in this notebook)
# It take left, middle and right part 
#   left part added to words suffix: _LEFT
#   right part added to words suffix: _RIGHT
#   middle part: no suffix
# Lenght for left,right,middle part has been tried (count of words) but didn't bring significant benefit
# Classificators which were tried: SVM, LogRegression (in code LogRegression is used)

def bag_of_words_featurizer_left_middle_right(kbt, corpus, feature_counter): 
    left_suffix = "_LEFT"
    right_suffix = "_RIGHT"
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
        for word in ex.left.split(' '):
            feature_counter[word + left_suffix] += 1
        for word in ex.right.split(' '):
            feature_counter[word + right_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
        for word in ex.left.split(' '):
            feature_counter[word + left_suffix] += 1
        for word in ex.right.split(' '):
            feature_counter[word + right_suffix] += 1

    return feature_counter

def test_bag_of_words_featurizer_left_middle_right(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel="rel", sbj="sbj", obj="obj")
    example1 = rel_ext.Example(entity_1='obj', entity_2='sbj', 
                               left='one two', mention_1='', middle='three four',
                               mention_2='', right='five', left_POS='', 
                               mention_1_POS='', middle_POS='', mention_2_POS='', right_POS='')
    crp = rel_ext.Corpus([example1])

    feature_counter = defaultdict(int)
  
    feature_counter['file_RIGHT'] += 5
    feature_counter['one_LEFT'] += 1
    feature_counter = bag_of_words_featurizer_left_middle_right(kbt, crp, feature_counter)
    expected = defaultdict(
        int, {'file_RIGHT': 5, 'one_LEFT': 2, 'three': 1, 'four': 1, 'two_LEFT': 1, 'five_RIGHT': 1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)


# Test that all works as expect
test_bag_of_words_featurizer_left_middle_right(corpus)

# Execution 
results = rel_ext.experiment(
splits,
train_split='train', 
test_split='dev',
featurizers=[bag_of_words_featurizer_left_middle_right]
)




/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.873      0.444      0.732        340       5716
author                    0.791      0.556      0.729        509       5885
capital                   0.462      0.189      0.359         95       5471
contains                  0.800      0.725      0.784       3904       9280
film_performance          0.817      0.640      0.774        766       6142
founders                  0.685      0.297      0.543        380       5756
genre                     0.680      0.200      0.459        170       5546
has_sibling               0.912      0.641      0.841        499       5875
has_spouse                0.816      0.596      0.760        594       5970
is_a                      0.708      0.489      0.650        497       5873
nationality               0.723      0.538      0.677        301       5677
parents     

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [28]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.


if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    rel_ext_data_home_test = os.path.join(rel_ext_data_home, 'bakeoff-rel_ext-test-data')
    bakeoff_results = rel_ext.experiment(splits,
                                         train_split='train', 
                                         test_split='dev',
                                         featurizers=[bag_of_words_featurizer_left_middle_right])
    rel_ext.bake_off_experiment(bakeoff_results, rel_ext_data_home_test)




/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.873      0.444      0.732        340       5716
author                    0.791      0.556      0.729        509       5885
capital                   0.462      0.189      0.359         95       5471
contains                  0.800      0.725      0.784       3904       9280
film_performance          0.817      0.640      0.774        766       6142
founders                  0.685      0.297      0.543        380       5756
genre                     0.680      0.200      0.459        170       5546
has_sibling               0.912      0.641      0.841        499       5875
has_spouse                0.816      0.596      0.760        594       5970
is_a                      0.708      0.489      0.650        497       5873
nationality               0.723      0.538      0.677        301       5677
parents     

In [ ]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    0.623


